# Insert triple for constituents related to objects

Run query:
```
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?s ?object
WHERE{
  Graph <https://example.com/ConXrefDetails/ConXrefDetails/assertion/c413cbfc/2024-02-12T06:55>{
    ?s <https://example.com/ConXrefs/vocab/ID> ?ID .
    ?s <https://example.com/ConXrefs/vocab/TableID> ?TableID .
    ?s <https://example.com/ConXrefs/vocab/TableID> ?TableID .
    {?s <https://example.com/ConXrefs/vocab/RoleTypeID> <https://example.com/RoleTypes/1>}
    UNION {?s <https://example.com/ConXrefs/vocab/RoleTypeID> <https://example.com/RoleTypes/2>} 
    UNION {?s <https://example.com/ConXrefs/vocab/RoleTypeID> <https://example.com/RoleTypes/5>} .
  }
  Graph <https://example.com/Bronbeek/Objects/Objects/assertion/bad21d24/2024-02-12T14:20>{
    ?object <https://example.com/Bronbeek/Objects/vocab/ObjectID> ?ID .
  }
}
```

add triple

In [ ]:
import pandas

df = pandas.read_csv("conxref_object_link.csv").dropna(axis=1)
df.columns

In [ ]:
import rdflib, os
from rdflib import Graph, URIRef
from rdflib.namespace import SKOS

def insert_links(df):
    g = Graph()

    try:
        for _ , row in df.iterrows():
            conxrefs = URIRef(str(row[0]).strip().replace('"', ''))
            object = URIRef(str(row[1]).strip().replace('"', ''))
            g.add((conxrefs, SKOS.related, object))
        
    finally:
        print(len(g))
        return g

In [ ]:
insert_links(df).serialize(destination='conxreftoobject_graph.ttl')

# Insert triple for provenance activity

run query:
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT ?event ?constituent ?object
WHERE{
    ?conxrefdetailID <https://example.com/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://example.com/ConXrefDetails/vocab/Prefix> ?event .
  	?conxrefdetailID <https://example.com/ConXrefDetails/vocab/RoleTypeID> ?RoleTypeID .
    ?conxrefdetailID <https://example.com/ConXrefDetails/vocab/ConXrefID> ?conxrefID .
  	?conxrefID skos:related ?object .
    FILTER (?RoleTypeID = <https://example.com/RoleTypes/2>).
}

add triples

In [2]:
import pandas

df = pandas.read_csv("provenance_activity.csv").dropna(axis=1)
df.columns

Index([' event', 'constituent', 'object'], dtype='object')

In [27]:
import rdflib, os, uuid
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import SKOS, RDF, RDFS

def insert_links(df):
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g = Graph()
    g.bind("crm", crm)

    try:
        for _ , row in df[:10].iterrows():
            unique_id = uuid.uuid4()

            event = str(row[0]).strip().replace('"', '')
            constituent = URIRef(str(row[1]).strip().replace('"', ''))
            object = URIRef(str(row[2]).strip().replace('"', ''))

            print(f"event:{event}\n constituent:{constituent}\n object: {object}\n")

            prov_activity = URIRef(u"www.example.com/Bronbeek/Provenance/1")
            acquisition = URIRef(u"www.example.com/Bronbeek/Acquisition/1")
            
            
            g.add(prov_activity, RDF.type, crm.E7_Activity)
            print("I am here")
            g.add(prov_activity, crm.P14_carried_out_by, constituent)
            g.add(prov_activity, crm.P2_has_type, URIRef(u"http://vocab.getty.edu/aat/300055863"))
            g.add(prov_activity, crm.P9_consists_of, acquisition)
            g.add(acquisition, RDF.type, crm.E8_Acquisition)
            g.add(acquisition, crm.P23_transferred_titled_from, constituent)
            g.add(acquisition, crm.P24_transferred_titled_of, object)
            
        
    finally:
        print(len(g))
        # g.add(URIRef(u"http://vocab.getty.edu/aat/300055863"), RDF.type, crm.E55_Type)
        # g.add(URIRef(u"http://vocab.getty.edu/aat/300055863"), RDFS.label, Literal("Provenance Activity"))
        return g

In [28]:
insert_links(df).serialize(destination='provenance_activity.ttl')

event:geschonken door
 constituent:https://example.com/Bronbeek/Constituents/2658
 object: https://example.com/Bronbeek/Objects/30966

0


/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_24034/1037965948.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  event = str(row[0]).strip().replace('"', '')
/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_24034/1037965948.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  constituent = URIRef(str(row[1]).strip().replace('"', ''))
/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_24034/1037965948.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior)

<Graph identifier=Ndeb3c363f7c04f8480455a08d8360f96 (<class 'rdflib.graph.Graph'>)>